# 이미지 특징 연습
*이 워크시트를 완성하고 제출하세요. (출력물과 워크시트에 포함되지 않은 코드들을 포함해서) 더 자세한 정보는 코스 웹사이트인 [숙제 페이지](http://vision.stanford.edu/teaching/cs231n/assignments.html)에서 볼 수 있습니다.*

우리는 입력된 이미지의 픽셀에 선형 분류기를 학습시켜 이미지 분류 작업에 적절한 성능을 얻을 수 있음을 알고있습니다.
이번 연습에서 우리는 단순 픽셀을 계산하기 위해 단순 픽셀(화소)이 아닌 특징을 통해 선형 분류기를 훈련시켜 우리의 분류 성능을 향상시킬 수 있음을 보일 것입니다.

이번 연습을 위한 모든 해야할 작업들은 이 notebook에서 수행됩니다.

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 기본 그래프 크기 설정
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# auto-reloading을 위한 외부 모듈
# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython를 보세요.
%load_ext autoreload
%autoreload 2

## 데이터 불러오기
이전 연습에서 처럼, 우리는 CIFAR-10 데이터를 불러올 것입니다.

In [ ]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
  # CIFAR-10 데이터를 불러옵니다.
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # 데이터 표본
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]

  return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

# 특징 추출하기
우리는 각 이미지 마다 그라데이션의 히스토그램(HOG)를 HSV색 공간에서의 색상 채널을 사용한 색상 히스토그램만큼 잘 계산할 것입니다. 우리는 우리의 마지막 특징 벡터를 각 이미지마다 HOG와 색상 히스토그램 특징 벡터를 이용하여 형성합니다.

강조하면, HOG 색상 정보를 무시하면서 이미지의 질감을 포착하고 색상 히스토그램은 질감을 무시하면서 입력된 이미지의 색상 나타낼 수 있습니다. 결과적으로, 우리는 두 가지를 동시에 사용하므로 한가지만 사용하는 것보다 더 효과적으로 작동할 것을 기대합니다. 이 가정을 증명하는 것은 보너스 단계에서 수행할만한 좋은 과제가 될 수 있습니다.

`hog_feature` 과 `color_histogram_hsv` 함수는 둘 다 하나의 이미지에서 그 이미지의 특징벡터를 반환하는 작업을 수행합니다. extract_features 함수는 이미지 집합과 특징 함수들의 목록을 가지고 각 이미지에 각각의 특징 함수를 평가하고 결과를 각 열이 하나의 이미지에 대한 모든 특징 벡터의 연결인 배열에 저장합니다.

In [ ]:
from cs231n.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# 전처리: 평균 특징 빼기
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# 전처리: 표준편차로 분리하기. 이것은 각 특징이 거의 같은 규모임을 보장합니다.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# 전처리: bias 차원 추가
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

## SVM을 특징에 대해서 훈련
이번 과제에서 작성한 멀티클래스 SVM 코드를 사용하여 위에서 추출된 특징을 이용해 SVM을 훈련합니다.
이 방법은 SVM을 단순픽셀을 이용하여 훈련시키는 것보다 더 좋은 결과를 얻을 수 있습니다.

In [ ]:
# Validation을 사용하여 학습 속도와 정규화 강도를 조정합니다.

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [1e5, 1e6, 1e7]

results = {}
best_val = -1
best_svm = None

pass
######################################################################################
# TODO:                                                                              #
# Validation을 사용하여 학습 속도와 정규화 강도를 조정합니다.                        #
# 이것은 SVM에서 했던 검증과 동일해야 합니다.                                        #
# 가장 잘 훈련된 분류기를 best_svm에 저장하세요.                                     #
# 아마 다른 개수의 색상 히스토그램안의 bin을 사용하여 해보고 싶을 수 있습니다.       #
# 아마 다른 개수의 색상 히스토그램안의 bin을 사용하여 해보고 싶을 수 있습니다.       #
# 만약 신중하다면, validation 세트에서 0.44에 근접한 정확도를 얻을 수 있을것 입니다. #
######################################################################################

pass
######################################################################################
#                                   코드의 끝                                        #
######################################################################################

# 결과를 출력합니다.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

In [ ]:
# 테스트 세트로 당신이 훈련시킨 SVM을 평가합니다.
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print test_accuracy

In [ ]:
# 알고리즘이 어떻게 작동하는지에 대한 직관을 얻기 위해 중요한 것은
# 알고리즘이 만드는 실수를 시각화 하는것 입니다.
# 이 시각화에서, 우리는 현재 시스템에서 잘못 분류된 이미지의 예제들을 보여줍니다.
# 첫 번째 열은 실제 "plane"은 아니지만 시스템이 "plane"으로 분류된 이미지를 보여줍니다.

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### 연습문제 1:
 잘못 분류된 결과에 대해 설명해보세요. 의미를 알 수 있나요?

## 이미지 특징의 신경망
이번 과제에서 우리는 단순 픽셀의 2-계층 신경망을 학습시키면 선형 분류기보다 성능이 더 향상됨을 배웠습니다. 이번 notebook에서 우리는 선형 분류기를 이미지 픽셀에 바로 적용하는 것보다 이미지에서 추출한 특징(feature)에 적용하는 것이 더 좋은 성능을 얻는 것을 알 수 있었습니다.

완성도를 위해, 우리는 이미지 특징의 신경망 또한 학습시켜보아야 합니다. 이 접근법은 이전의 모든 방법보다 더 뛰어날 것입니다: 테스트 세트에 대해 55%이상의 분류 정확도를 쉽게 달성할 수 있어야합니다; 우리의 최고의 모델은 60%의 분류 정확도를 달성했습니다.

In [ ]:
print X_train_feats.shape

In [ ]:
from cs231n.classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO:                                                                        #
# 이미지 특징으로 2-계층 신경망 학습시키기.                                    #
# 이전 섹션처럼 다양한 변수들을 교차검증하기.                                  #
# 최고의 모델을 best_net 변수에 저장하기.                                      #
################################################################################
pass
################################################################################
#                                   코드의 끝                                  #
################################################################################

In [ ]:
# 당신의 신경망 분류기를 테스트 세트로 실행시켜 보세요.
# 55% 이상의 정확도를 얻을 수 있어야 합니다.

test_acc = (net.predict(X_test_feats) == y_test).mean()
print test_acc

# 보너스: 당신만의 특징을 디자인해보세요!

간단한 이미지 특징이 분류기의 성능을 향상시킬 수 있음을 배웠습니다. 지금까지 우리는 HOG와 색상 히스토그램을 통해 시도해봤지만 다른 종류의 특징들은 분류 성능을 더 향상시킬 수 있습니다.

보너스 포인트를 위해, 새로운 종류의 특징을 디자인하고 적용하고 CIFAR-10의 이미지 분류에 사용해 보세요. 당신의 특징이 어떻게 작동하고 왜 그러한 특징이 이미지 분류에 효과적으로 작동할 것이라 생각했는데 설명해보세요. 이 notebook에서 적용해보고, 임의의 hyperparameters로 교차 검증 하고 HOG + 색상 히스토그램 기준과 성능을 비교해보세요.

# 보너스: 뭔가 더 해보세요!
이번 과제에서 제공된 자료와 코드를 사용하여 흥미로운 도전을 해보세요. 과제를 하면서 다른 의문점이 생겼나요? 과제를 하면서 머리에서 참신한 생각이 떠올랐나요? 당신을 보여줄 수 있는 기회입니다!